# Balance in-house and external production of pasta

This tutorial includes information and data that you need to set up IBM Decision Optimization CPLEX Modeling for Python (DOcplex), build a mathematical programming (MP) model, and solve the model with IBM ILOG CPLEX Optimizer.


When you finish this tutorial, you'll have a foundational knowledge of _Prescriptive Analytics_.

>This notebook is part of [Prescriptive Analytics for Python](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html).

>You will need a valid subscription to Decision Optimization on Cloud ([here](https://developer.ibm.com/docloud)) or a local installation of CPLEX Optimizers. 

Some familiarity with Python is recommended. This notebook runs on Python 2.

## Describe the business problem

This notebook describes how to use CPLEX Modeling for Python to balance in-house and external production of pasta. The goal is to minimize the production cost for some pasta products and ensure that the customers' demand for the products is satisfied. 
The following scenarios are considered: 

   * Each product can be either produced in-house, by the company itself, or outsourced to an external company at a higher cost.
   * The in-house production is constrained by the resources, such as flour and eggs, while the external production is considered unlimited.

To investigate possible solutions, a mathematical programming (MP) model is built that declares the products and resources, and specifies the data and variables.

## How  decision optimization can help 

Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes.  It considers specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes. For more information, go to the [Prescriptive analytics](https://ibm.com/software/products/sr/category/prescriptive-analytics) website.

## Table of contents

1. [Download the library](#download_library)<br>
2. [Set up the prescriptive engine](#set_prescriptive_engine)<br>
3. [Model the data](#model_data)<br>
4. [Set up the prescriptive model](#set_prescriptive_model)<br>
    4.1 [Create the docplex model](#set_prescriptive_model_docplex)<br>
    4.2 [Define the decision variables](#set_prescriptive_model_variables)<br>
    4.3 [Define the business constraints](#set_prescriptive_model_constraints)<br>
    4.4 [Specify the objective](#set_prescriptive_model_objective)<br>
    4.5 [Solve the model](#set_prescriptive_model_solve)<br> 
5. [Investigate the solution and run a sample analysis](#investigate_solution)<br>

<a id="download_library"></a>
## 1. Download the library

Run the following code to install the Decision Optimization CPLEX modeling library. The DOcplex library contains two modeling packages: mathematical programming (docplex.mp) package and constraint programming (docplex.cp) package.

In the following code, `real.prefix` is used to ensure that the script is not running inside a virtualenv environment.


In [1]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex 
    else:
        !pip install --user docplex

<a id="set_prescriptive_engine"></a>
## 2. Set up the prescriptive engine

* Subscribe to your private Cloud offering, or to Decision Optimization on Cloud solve service [here](https://developer.ibm.com/docloud).

* Get the Decision Optimization service URL and personal API key and enter them in the cell below, replacing the `None` values. Enclose the values in quotation marks (" "). For more information, go [here](https://developer.ibm.com/docloud/documentation/decision-optimization-on-cloud/api-key/).



In [2]:
url = None
key = None

<a id="model_data"></a>
## 3. Model the data

The data includes descriptions (demand, in-house and external production costs, resource consumption) of the different pasta products (kluski, capellini, fettucine), and the capacity of the resources (flour, eggs). Resources are a list of simple tuples (name, capacity).

In [3]:
products = [("kluski", 100, 0.6, 0.8),
            ("capellini", 200, 0.8, 0.9),
            ("fettucine", 300, 0.3, 0.4)]

resources = [("flour", 20),
             ("eggs", 40)]

consumptions = {("kluski", "flour"): 0.5,
                ("kluski", "eggs"): 0.2,
                ("capellini", "flour"): 0.4,
                ("capellini", "eggs"): 0.4,
                ("fettucine", "flour"): 0.3,
                ("fettucine", "eggs"): 0.6}

The data is very simple and ready to use without any cleansing or refactoring.

<a id="set_prescriptive_model"></a>
## 4. Set up the prescriptive model

Start with viewing the environment information.

In [4]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python is present, version is 2.7.11
* docplex is present, version is (2, 3, 44)
* CPLEX wrapper is not available


<a id="set_prescriptive_model_docplex"></a>
### 4.1 Create the docplex model  

CPLEX Modeling for Python is used to build a mixed integer programming (MIP) model for this problem. In an MIP model, one or more variables must take integer solution values.

The model defines the decision variables, the business constraints, and the objective.

In [5]:
from docplex.mp.model import Model
mdl = Model(name="pasta")

<a id="set_prescriptive_model_variables"></a>
###  4.2 Define the decision variables

The decision variables are:
* `inside_vars` for in-house production
* `outside_vars` for external production 

In [6]:
inside_vars = mdl.continuous_var_dict(products, name='inside')
outside_vars = mdl.continuous_var_dict(products, name='outside')

<a id="set_prescriptive_model_constraints"></a>
### 4.3 Define the business constraints 

The constraints should address the following scenarios:
   * Each pasta product can be produced either in-house or externally. 
   * The in-house production is constrained by the availability of resources, while the external production is considered unlimited



In [7]:
# Demand satisfaction
mdl.add_constraints((inside_vars[prod] + outside_vars[prod] >= prod[1], 'ct_demand_%s' % prod[0]) for prod in products)

# Resource capacity
mdl.add_constraints((mdl.sum(inside_vars[p] * consumptions[p[0], res[0]] for p in products) <= res[1], 'ct_res_%s' % res[0]) for res in resources)

mdl.print_information()

Model: pasta
 - number of variables: 6
   - binary=0, integer=0, continuous=6
 - number of constraints: 5
   - linear=5
 - parameters: defaults


<a id="set_prescriptive_model_objective"></a>
### 4.4 Specify the objective 

The objective is to minimize the production cost for a number of pasta products, and at the same time, ensure that the customers' demand is satisfied.

In the following code, `total_inside_cost` represents the total in-house production cost, and `total_outside_cost` represents the total external production cost.

In [8]:
total_inside_cost = mdl.sum(inside_vars[p] * p[2] for p in products)

total_outside_cost = mdl.sum(outside_vars[p] * p[3] for p in products)

mdl.minimize(total_inside_cost + total_outside_cost)

<a id="set_prescriptive_model_solve"></a>
### 4.5 Solve the model

Now, the model can be solved using the Decision Optimization solve service `Model.solve()`.

The following cell can also be solved using your local CPLEX optimizer, if you have one installed and added to your `PYTHONPATH` variable. 
If you do not have CPLEX installed, enter your DOcplexcloud credentials in the `key` and `url` fields to solve on DOcplexcloud.

In [9]:
mdl.solve(url=url, key=key)

docplex.mp.solution.SolveSolution(obj=372,values={inside_('kluski', 100,..

<a id="investigate_solution"></a>
## 5. Investigate the solution and run a sample analysis

In [10]:
obj = mdl.objective_value

print("* Production model solved with objective: {:g}".format(obj))
print("* Total inside cost=%g" % total_inside_cost.solution_value)
for p in products:
    print("Inside production of {product}: {ins_var}".format(product=p[0], ins_var=inside_vars[p].solution_value))
print("* Total outside cost=%g" % total_outside_cost.solution_value)
for p in products:
    print("Outside production of {product}: {out_var}".format(product=p[0], out_var=outside_vars[p].solution_value))

* Production model solved with objective: 372
* Total inside cost=24
Inside production of kluski: 40.0
Inside production of capellini: 0
Inside production of fettucine: 0
* Total outside cost=348
Outside production of kluski: 60.0
Outside production of capellini: 200.0
Outside production of fettucine: 300.0


The proposed solution suggests in-house production of 40 units of kluski, and external production of 60 units of kluski, 200 units of capellini, and 300 units of fettucine.

## Summary

You've learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to create a mathematical programming model and solve it with IBM Decision Optimization on Cloud.

## References
* [CPLEX Modeling for Python documentation](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* [Decision Optimization documentation](http://datascience.ibm.com/docs/content/DO/DOinDSX.html)
* For help with DOcplex, or to report a defect, go [here](https://developer.ibm.com/answers/smartspace/docloud).
* Contact us at dofeedback@wwpdl.vnet.ibm.com.

<div class="alert alert-block alert-info"> Note: To save resources and get the best performance please use the code below to stop the kernel before exiting your notebook.</div>

In [11]:
%%javascript
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>

<hr>
Copyright &copy; IBM Corp. 2017. Released as licensed Sample Materials.